# Семинар 4

На этом семинаре мы:

- решим задачу классификации с помощью рекуррентной нейронной сети

Используем датасет с именами и будем определять, из какого языка произошло данное имя. Для этого построим рекуррентную сеть, которая работает с именами на уровне символов. А именно, на вход сети подается набор символов (имя), и на каждом шаге на выходе из сети оказываются предсказание и скрытое состояние, которое используется на следующем шаге. Предсказание на последнем шаге будет браться за итоговое, то есть мы получим прогноз сети, к какому классу принадлежит данное имя.

Ссылка для скачивания: https://download.pytorch.org/tutorial/data.zip

In [1]:
! wget https://download.pytorch.org/tutorial/data.zip -O data.zip && unzip -qq ./data.zip

--2021-11-08 06:33:10--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.249.93.46, 13.249.93.56, 13.249.93.14, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.249.93.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.09s   

2021-11-08 06:33:10 (31.3 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [ ]:
! ls ./data/names

Arabic.txt   English.txt  Irish.txt	Polish.txt	Spanish.txt
Chinese.txt  French.txt   Italian.txt	Portuguese.txt	Vietnamese.txt
Czech.txt    German.txt   Japanese.txt	Russian.txt
Dutch.txt    Greek.txt	  Korean.txt	Scottish.txt


### Обработка данных

Для начала приведем текстовые файлы с именами к удобному формату, так, что на выходе получится словарь язык-список имен. В код можно особо не вникать.

In [4]:
! head ./data/names/English.txt

Abbas
Abbey
Abbott
Abdi
Abel
Abraham
Abrahams
Abrams
Ackary
Ackroyd


In [6]:
from io import open
import numpy as np
import os

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

names_path = 'data/names/'

print(os.listdir(names_path))


# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in os.listdir(names_path):
    category = filename.split('.')[0]
    all_categories.append(category)
    lines = readLines(names_path + filename)
    category_lines[category] = lines

n_categories = len(all_categories)

Slusarski
['Chinese.txt', 'French.txt', 'Korean.txt', 'Polish.txt', 'German.txt', 'Dutch.txt', 'Russian.txt', 'Italian.txt', 'Vietnamese.txt', 'Czech.txt', 'Spanish.txt', 'English.txt', 'Japanese.txt', 'Arabic.txt', 'Irish.txt', 'Scottish.txt', 'Portuguese.txt', 'Greek.txt']


In [7]:
n_categories

18

In [8]:
print(np.random.choice(category_lines['Korean'], size=7, replace=False))
print(np.random.choice(category_lines['Russian'], size=7, replace=False))

['Ha' 'Hong' 'Hung' 'Kwak' 'Song' 'Mo' 'Yoon']
['Dubenkov' 'Velikanov' 'Yanovich' 'Hihich' 'Vikhirev' 'Jigunov' 'Dublin']


In [9]:
len(all_letters)

57

Конвертируем имена в тензоры для того, чтобы их можно было подать на вход сети. Для этого векторизуем каждую букву бинарным вектором из нулей и единиц с 1 на позиции, соответствующей индексу буквы в алфавите. Например, `"c" -> (0 0 1 0 0 ...)`. Таким образом, имя превратится в тензор размера `количество символов x величина батча x количество букв в алфавите`. В данном случае будем использовать 1 батч, букв в алфавите всего 57, поэтому итоговая размерность будет `количество символов x 1 x 57`.

In [10]:
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'"

In [11]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.index(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    result = torch.zeros(1, len(all_letters))
    result[:, letterToIndex(letter)] = 1
    return result

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    result = [letterToTensor(letter) for letter in line]
    result = torch.cat(result)
    return result[:, None, :]

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [12]:
lineToTensor('Jones')

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0

### Построение модели

![image](https://i.imgur.com/Z2xbySO.png)

Реализуем простейшую однослойную RNN.

In [39]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.h2o = nn.Linear(hidden_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)


    def forward(self, input, hidden):
        inp = torch.cat((input, hidden), -1)
        next_hidden = self.relu(self.i2h(inp))
        output = self.h2o(hidden)
        return output, next_hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

Проверим, что все корректно работает: выходы классификатора должны быть лог-вероятностями (чем больше, тем вероятнее категория).

In [40]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

output.shape

torch.Size([1, 18])

In [41]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)
print(torch.exp(output).sum())

tensor([[ 0.0843, -0.0291,  0.0019, -0.0487, -0.0758, -0.0569,  0.0352, -0.0720,
         -0.0582,  0.0625, -0.0501, -0.0480, -0.0707,  0.0530, -0.0770, -0.0774,
          0.0041,  0.0646]], grad_fn=<AddmmBackward>)
tensor(17.6724, grad_fn=<SumBackward0>)


### Обучение

Чтобы интерпретировать выход модели, напишем функцию, переводящую лог-вероятности в категорию.

In [42]:
output

tensor([[ 0.0843, -0.0291,  0.0019, -0.0487, -0.0758, -0.0569,  0.0352, -0.0720,
         -0.0582,  0.0625, -0.0501, -0.0480, -0.0707,  0.0530, -0.0770, -0.0774,
          0.0041,  0.0646]], grad_fn=<AddmmBackward>)

In [43]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Chinese', 0)


Функция для получения случайного объекта из обучающей выборки.

In [91]:
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'"

In [95]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])[:32]
    while len(line) < 32:
        line += ";"

    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = French / line = Cote;;;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Scottish / line = Thomson;;;;;;;;;;;;;;;;;;;;;;;;;
category = Korean / line = Chi;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Dutch / line = Amelsvoort;;;;;;;;;;;;;;;;;;;;;;
category = Czech / line = Zaruba;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Arabic / line = Mustafa;;;;;;;;;;;;;;;;;;;;;;;;;
category = French / line = Leroy;;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Arabic / line = Sarraf;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Scottish / line = Shaw;;;;;;;;;;;;;;;;;;;;;;;;;;;;
category = Japanese / line = Kawabata;;;;;;;;;;;;;;;;;;;;;;;;



Теперь перейдем непосредственно к обучению. Для этого напишем функцию, в ходе которой:

- на вход поступают объект (имя) и класс (язык)
- инициализируются скрытые состояния (нулями)
- forward-pass: считывается каждый символ и скрытое состояние для него сохраняется для следующего символа
- считается итоговое предсказание
- считается значение функции потерь (loss)
- backward-pass и обновление весов
- возвращаются loss и качество предсказания (accuracy)

In [ ]:
nn.RNN(), nn.LSTM(), nn.GRU()

In [99]:
criterion = nn.CrossEntropyLoss()
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        self.input_size = input_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)


    def forward(self, input, hidden):
        # input.shape = [N, L, H]
        _, h_n = self.rnn(input)
        output = self.linear(h_n)
        return output

    def initHidden(self):
        return torch.zeros(self.hidden_size)

n_hidden = 128

def train(category, category_tensor, line_tensor, optimizer):

    rnn.zero_grad()
    line_tensor = line_tensor.permute(1, 0, 2)

    # for i in range(line_tensor.shape[0]):
    output = rnn(line_tensor, hidden)
    # print(output.shape, category_tensor.shape)
    loss = criterion(output[:, 0], category_tensor)
    loss.backward()
    optimizer.step()

    acc = (categoryFromOutput(output)[0] == category)

    return loss.item(), acc

from tqdm import trange
import torch.nn.functional as F
import matplotlib.pyplot as plt


n_iters = 50000
plot_every = 1000

current_loss = 0
all_losses = []
current_acc = 0
all_accs = []

n_hidden = 128

rnn = RNN(n_letters, n_hidden, n_categories).cuda()
opt = torch.optim.RMSprop(rnn.parameters(), lr=0.001)
batch_size = 64

for iter in trange(1, n_iters + 1):
    X_batch = []
    y_batch = []
    for i in range(batch_size):
        category, line, category_tensor, line_tensor = randomTrainingExample()
        _, _, category_tensor, line_tensor = category, line, category_tensor, line_tensor
        category_tensor = category_tensor.cuda()
        line_tensor = line_tensor.cuda()
        X_batch.append(line_tensor)
        y_batch.append(category_tensor)
    
    
    X_batch = torch.cat(X_batch, dim=1)
    y_batch = torch.cat(y_batch).view(-1)
    loss, acc = train(category, category_tensor, line_tensor, opt)
    current_loss += loss
    current_acc += acc

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0
        all_accs.append(current_acc / plot_every)
        current_acc = 0

  5%|▍         | 2352/50000 [01:35<32:19, 24.57it/s]


KeyboardInterrupt: ignored

In [90]:
# F.pad(torch.ones(3, 3), (4, 4, 4, 4), value=0)
line_tensor.shape

torch.Size([7, 1, 57])

In [56]:
plt.figure()
plt.title("Loss")
plt.plot(range(plot_every, n_iters + 1, plot_every), all_losses)
plt.grid()
plt.show()

plt.figure()
plt.title("Accuracy")
plt.plot(range(plot_every, n_iters + 1, plot_every), all_accs)
plt.grid()
plt.show()

ImportError: ignored

<Figure size 432x288 with 1 Axes>

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [100]:
all_accs

[0.062, 0.066]

Теперь обучим построенную модель на случайных объектах из обучающей выборки и построим графики значений функции потерь и качества. На каждом тысячном шагу будем ставить отметку, соответствующую среднему значению на предыдущих `1000` объектах.

In [46]:
! pip uninstall -y matplotlib

### Результат

In [ ]:
! pip install --user --upgrade matplotlib

    100% |████████████████████████████████| 12.5MB 1.5MB/s ta 0:00:011  60% |███████████████████▌            | 7.6MB 6.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /Users/sgord1/anaconda3/lib/python3.6/site-packages (from matplotlib) (2.2.2)
Requirement not upgraded as not directly required: kiwisolver>=1.0.1 in /Users/sgord1/anaconda3/lib/python3.6/site-packages (from matplotlib) (1.0.1)
Requirement not upgraded as not directly required: numpy>=1.11 in /Users/sgord1/anaconda3/lib/python3.6/site-packages (from matplotlib) (1.14.5)
Requirement not upgraded as not directly required: cycler>=0.10 in /Users/sgord1/anaconda3/lib/python3.6/site-packages (from matplotlib) (0.10.0)
Requirement not upgraded as not directly required: python-dateutil>=2.1 in /Users/sgord1/anaconda3/lib/python3.6/site-packages (from matplotlib) (2.7.3)
Requirement not upgraded as not directly required: setuptools in /Users/sgord1/anaconda3/lib/pyt

Сделаем предсказание сети для `10000` случайных объектов. Для визуализации результата построим матрицу ошибок для языков.

In [104]:
import matplotlib.ticker as ticker

# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()[None]

    # for i in range(line_tensor.size()[0]):
    output = rnn(line_tensor.cuda(), hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure(figsize=(16,6))
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

ValueError: ignored

Время поэкспериментировать с предсказаниями!

In [102]:
def predict(input_line, n_predictions=5):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('asdasdasdasd')


> asdasdasdasd


RuntimeError: ignored

### To do

- измените параметр `learning_rate` внутри оптимизатора. Что происходит, когда он слишком большой? А слишком маленький?

- добавьте больше слоев!

- решите эту же задачу с помощью LSTM и GRU блоков (`nn.LSTM`, `nn.GRU`)

- **(*)** постройте более сложные модели на основе уже испробованных - например, двусторонние (bidirectional) LSTM и GRU